In [1]:
import os
import sys
from pathlib import Path

import pendulum
import numpy as np
import porepy as pp


sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import GTS as gts
import GTS.test.util as test_util



/home/haakon/porepy/src/porepy/viz/exporter.py:928: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


In [2]:
import logging
logging.basicConfig(level="INFO")

In [3]:
def setup_simple_gb(**kwargs):
    """ Setup simple grid bucket
    
    Keyword arguments used for
        length_scale,
        scalar_scale,
        path_tail (Default: "test_1"),
        setup_loggers (Default: False),
        
    """
    length_scale = kwargs.get("length_scale", 1)
    scalar_scale = kwargs.get("scalar_scale", 1)
    
    # Storage path details
    path_tail = kwargs.get("path_tail", "test_1")
    name = "test_parameter_scaling_on_regular_grid_ipynb"
    date = pendulum.now().format("YYMMDD")
    _folder_root = f"{name}/{date}/{path_tail}/ss{scalar_scale}ls{length_scale}"
    
    # Parameter setup
    
    # Size of domain - scaled by length scale.
    physdims = np.array([10, 10, 10]) / length_scale

    params = {
            "mesh_args":
                {None},
            "bounding_box":
                {"xmin": 0, "xmax": physdims[0],
                 "ymin": 0, "ymax": physdims[1],
                 "zmin": 0, "zmax": physdims[2],},
            "shearzone_names":
                ["S1_1"],
            "length_scale":
                length_scale,
            "scalar_scale":
                scalar_scale,
            "_gravity_bc_p": True,
            "_gravity_src": True,
            "_gravity_bc": True,
        }
    
    setup_loggers = kwargs.get("setup_loggers", False)

    setup = test_util.prepare_setup(
            model=gts.ContactMechanicsBiotISC,
            path_head=_folder_root,
            params=params,
            prepare_simulation=False,
            setup_loggers=setup_loggers,
        )
    
    ### Set up simple grid bucket with one fracture ###
    
    # Grid is 2x2x2 cells, with a fracture in the lower middle part of the grid.
    #  _ _
    # |   |
    # |_|_|

    # Frac point coordinates, scaled by length scale
    # The fracture lies in the yz-plane.
    f_1 = np.array([
        [5, 5, 5, 5],
        [0, 0, 5, 5],
        [0, 5, 5, 0]
    ]) / length_scale
    
    # Number of cells in each dimension.
    n_cells = np.array([2, 2, 2])

    gb = pp.meshing.cart_grid(
        fracs=[f_1],
        nx=n_cells,
        physdims=physdims,
    )
    
    # Assign the simple grid to the grid bucket
    setup.set_grid(gb)
    
    run_model = kwargs.get("run_model", True)
    if run_model:
        nl_options = {
            "max_iterations": 20,
        }
        pp.run_time_dependent_model(setup, nl_options)
    
    return setup

# Assert solutions

In [4]:
def assert_solutions(s1, s2, dim):
    """ Assert that solutions scale correctly
    
    Parameters
    ----------
    s1, s2 : ContactMechanicsBiotISC
        Setups
    dim : int
        which grid dimension to extract solutions over
    """
    print(f"Checking solutions for two setups on grid of dimension: {dim}")
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        d = gb.node_props(g)
        state = d[pp.STATE]
        
        ### Quantities ###
        cell_centers = g.cell_centers * ls
        cell_volumes = g.cell_volumes * (ls)**dim
        face_areas = g.face_areas * (ls)**(dim-1)
        p = state['p'] * ss
        p_exp = state['p_exp']
        
        Nd = s.Nd
        # Matrix displacement
        if dim == Nd:
            u = state['u'] * ls
            tangential_traction = [0]
            normal_traction = [0]
            u_t_norm = [0]
            u_n = [0]
        # Fracture displacement in global coordinates
        elif dim == Nd - 1:
            # Fetch the mortar grid and data
            e, de = [i for i in gb.edges()][0]
            
            # Displacement
            mg = de['mortar_grid']
            mortar_u = de['state']['mortar_u']
            u = (mg.mortar_to_slave_avg(nd=Nd) 
                 * mg.sign_of_mortar_sides(nd=Nd)
                 * mortar_u) * ls
            
            # Displacement (local coords)
            u_local = s.reconstruct_local_displacement_jump(de, from_iterate=False) *ls
            u_t_norm = np.linalg.norm(u_local[:2, :], axis=0)
            u_n = u_local[2, :]
            
            # Traction
            # recall: traction is represented in local coordinates directly.
            proj = de['tangential_normal_projection']
            proj.tangential_basis[:,:,0]
            traction = d['state']['contact_traction']
            tangential_traction_local = traction[:-1]
            normal_traction = traction[-1]
            tangential_traction = proj.tangential_basis[:,:,0].dot(tangential_traction_local)
            # upscale the scaled result
            normal_traction = normal_traction * ss * ls**(Nd-1)
            tangential_traction = tangential_traction * ss * ls**(Nd-1)
        

        return (cell_centers, cell_volumes, face_areas, p, p_exp, u, 
                normal_traction, tangential_traction, u_t_norm, u_n)
    
    (cell_centers1, cell_volumes1, face_areas1, p1, p_exp1, u1, 
     normal_traction1, tangential_traction1, u_t_norm1, u_n1
    ) = fetch_values_from_setup(s1, dim)
    
    (cell_centers2, cell_volumes2, face_areas2, p2, p_exp2, u2, 
     normal_traction2, tangential_traction2, u_t_norm2, u_n2
    ) = fetch_values_from_setup(s2, dim)

    ### Assertions ###
    # Assert that each geometric index corresponds to one another
    assert np.allclose(cell_centers1, cell_centers2)
    
    # Assert that all volumes and areas are the same
    assert np.allclose(cell_volumes1, cell_volumes2)
    assert np.allclose(face_areas1, face_areas2)
    
    ## Non-zero on all grids ##
    # Assert that the pressure results are equal
    p_true = np.allclose(p1, p2)
#     assert p_true
    print(f"p: {p_true}")
    
    # Assert that the exported results are upscaled correctly
    p_exp_true = np.allclose(p_exp1, p_exp2)
#     assert p_exp_true
    print(f"p_exp: {p_exp_true}")
    
    # Assert that the displacement results are equal
    u_true = np.allclose(u1, u2)
#     assert u_true
    print(f"u: {u_true}")
    
    # Assert that norm of tangential displacement is equal
    u_t_norm_true = np.allclose(u_t_norm1, u_t_norm2)
    print(f"u_t_norm: {u_t_norm_true}")
    # Assert that norm displacement is equal
    u_n_true = np.allclose(u_n1, u_n2)
    print(f"u_n: {u_n_true}")


    # Traction
    n_traction_true = np.allclose(normal_traction1, normal_traction2)
#     assert n_traction_true
    print(f"normal_traction: {n_traction_true}")
    t_traction_true = np.allclose(tangential_traction1, tangential_traction2)
#     assert t_traction_true
    print(f"tangential_traction: {t_traction_true}")
    
    assert (p_true and p_exp_true and u_true and n_traction_true and t_traction_true
            and u_t_norm_true and u_n_true
           )
    

# Check scaling of parameters 

In [5]:
def check_parameter_scaling(s1, s2, dim):
    """ Test parameter scaling for two setups"""
    print(f"Checking parameters for two setups on grid of dimension: {dim}")
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        data = gb.node_props(g)
        params_u = data['parameters']['mechanics']
        params_p = data['parameters']['flow']
        
        neu_p = params_p['bc'].is_neu
        dir_p = params_p['bc'].is_dir
        
        # Compute unscaled quantities: 
        # MECHANICS
        if dim == s.Nd: # On rock matrix
            neu_u = (params_u['bc'].is_neu).ravel(order='F')
            dir_u = (params_u['bc'].is_dir).ravel(order='F')
            mech = {
                'bc_values': {
                    'dir': 
                    params_u['bc_values'][dir_u].reshape((3, -1), order='F') * ls, # [L]
                    'neu': 
                    (params_u['bc_values'][neu_u].reshape((3, -1), order='F') 
                     * ss * ls**(dim-1)
                    ), # [P L^(dim-1)]
                },
                'source':
                    params_u['source'] * ss * ls**(dim-1), # [P L^(dim-1)]
                'biot_alpha': # [ ]
                    params_u['biot_alpha'], 
                'mu':
                    params_u['fourth_order_tensor'].mu * ss, # [P]
                'lambda':
                    params_u['fourth_order_tensor'].lmbda * ss, # [P]
            }
        elif dim == s.Nd-1: # On fracture
            mech = params_u # We're only interested in the friction coefficient
        
        # On the fracture, terms are weighed by the specific volume!
        sv_scale = np.power(ls, s.Nd - dim)
        # FLOW
        flow = {
            'bc_values': {
                'dir': params_p['bc_values'][dir_p] * ss, # [P]
                'neu': params_p['bc_values'][neu_p] * ls**dim, # [L^(dim)]
            },
            'mass_weight':  
                params_p['mass_weight'] * sv_scale / ss,  # [L^(Nd-dim) / P]
            'source':  # -- integrated source is of dim Nd regardless of 
                       #    ambient dimension!!
                params_p['source'] * ls**s.Nd, # [L^Nd] 
            'biot_alpha': # [ ]
                params_p['biot_alpha'],
            'kxx':  # k/mu * aperture^(Nd-dim) : [L^(Nd-dim) * L^2 / P]
                params_p['second_order_tensor'].values[0, 0, :] * ls**2 / ss * sv_scale,
        }
        return mech, flow
    
    mech1, flow1 = fetch_values_from_setup(s1, dim)
    mech2, flow2 = fetch_values_from_setup(s2, dim)
    
    # -- ASSERTIONS --
    # Mechanics
    if dim == s1.Nd:
        # BC:
        assert np.allclose(mech1['bc_values']['dir'], mech2['bc_values']['dir'])
        assert np.allclose(mech1['bc_values']['neu'], mech2['bc_values']['neu'])
        # Source
        assert np.allclose(mech1['source'], mech2['source'])
        # Physical parameters
        assert np.allclose(mech1['biot_alpha'], mech2['biot_alpha'])
        assert np.allclose(mech1['mu'], mech2['mu'])
        assert np.allclose(mech1['lambda'], mech2['lambda'])
    elif dim == s1.Nd-1:
        assert np.allclose(mech1['friction_coefficient'], mech2['friction_coefficient'])
    
    # Flow
    # BC:
    assert np.allclose(flow1['bc_values']['dir'], flow2['bc_values']['dir'])
    assert np.allclose(flow1['bc_values']['neu'], flow2['bc_values']['neu'])
    # Source
    assert np.allclose(flow1['source'], flow2['source'])
    # Physical parameters
    assert np.allclose(flow1['biot_alpha'], flow2['biot_alpha'])
    assert np.allclose(flow1['kxx'], flow2['kxx'])
    
def check_mortar_parameter_scaling(s1, s2):
    def fetch_values_from_setup(s):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        
        for e, d in gb.edges():
            k = d['parameters']['flow']['normal_diffusivity'] * ls/ss
        return k
    
    k1 = fetch_values_from_setup(s1)
    k2 = fetch_values_from_setup(s2)
    
    assert np.allclose(k1,k2)

In [6]:
def checks(T1, T2):
    ## Assert parameters on 3D grid ##
    check_parameter_scaling(T1, T2, 3)

    ## Assert parameters on 2D grid ##
    check_parameter_scaling(T1, T2, 2)

    #

    ## Assert solutions on 3D grid ##
    assert_solutions(T1, T2, 3)

    ## Assert solutions on 2D grid ##
    assert_solutions(T1, T2, 2)

# Run with unscaled variables
I.e., the following (default) scalings apply:\
`length_scale = 1`\
`scalar_scale = 1`

In [7]:
unscl = setup_simple_gb()

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1ls1
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:05+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01491
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00083
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed time
   

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1ls1


# Next, re-run with scaled variables
We apply the following scalings:\
`length_scale = 1`\
`scalar_scale = 1 * pp.MEGA`

This isolates testing for the `scalar_scale`.

In [8]:
scl = setup_simple_gb(
    length_scale=1,
    scalar_scale=1*pp.MEGA,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls1
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:16+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01570
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00056
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed 

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls1


In [9]:
# ONLY SCALAR SCALE

check_mortar_parameter_scaling(scl, unscl)

## Assert parameters on 3D grid ##
check_parameter_scaling(scl, unscl, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl, unscl, 2)

#

## Assert solutions on 3D grid ##
assert_solutions(scl, unscl, 3)

## Assert solutions on 2D grid ##
assert_solutions(scl, unscl, 2)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


### Sjekk slip i globale koordinater!

* Sjekk med mindre skalering (1e8, 1e6)
* Introduser lengdeskalering


* Du kan regne ut analytisk om du vil ha slip
Hvilke skjærkrefter har du. Bruk MC for å regne om du får slip.

Du kan regne betingelser for å akkurat ha slip eller akkurat ikke ha slip.

Rotasjon av stress kan gjøre at vi får slip.

------------
# Compare moderately scaled solutions
We apply the following scalings:\
`length_scale = 1`\
`scalar_scale = 1e8`

We aim to compare the solutions with scalar scale 1e8 and 1e6. Hopefully these should be very similar.

In [10]:
scl_mod = setup_simple_gb(
    length_scale=1,
    scalar_scale=1e8,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls1
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:24+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01500
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00059
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapse

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls1


In [11]:
# ONLY SCALAR SCALE

## Assert parameters on 3D grid ##
check_parameter_scaling(scl, scl_mod, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl, scl_mod, 2)

#

## Assert solutions on 3D grid ##
assert_solutions(scl, scl_mod, 3)

## Assert solutions on 2D grid ##
assert_solutions(scl, scl_mod, 2)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


# Mulige problem (fra veilednignsmøte 24.03.2020)

* Kan hende det ikke er lengdeskalering på randkrav
* Slå av gravitasjonseffekter (sett til 0)
   - Slå av vann (Dirichlet=0) og gravitasjon.
   - Problemet kan være drevet av anisotrop bakgrunnsstress.
  
Merk: Problemet er nok ikke drevet av væske. Hvis det er satt opp riktig vil det være en likevekt mellom randkrav og kildeledd. Dermed vil ikke det være utgangspunktet for feilene våre.

Merk2: Feilen i trykket er for stort til at det handler om dårlig kondisjonering av problemet. Det er heller drevet av feil i grunnleggende skalering av -- rand, kilde, parameter.

In [12]:
hard = setup_simple_gb(
    length_scale=.1,
    scalar_scale=1e6,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls0.1
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:34+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01803
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00079
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapse

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls0.1


In [13]:
assert_solutions(scl_mod, hard, 3)
assert_solutions(scl_mod, hard, 2)

Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


# Optimize for best condition number
* We assume that `length_scale` and `scalar_scale` are independent and have separate minima

In [14]:
best = setup_simple_gb(
    length_scale=.1,
    scalar_scale=1e8,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls0.1
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:43+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01637
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00094
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellap

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls0.1


In [15]:
best2 = setup_simple_gb(
    length_scale=.1  * .5,
    scalar_scale=1e8 * 1,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls0.05
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:28:52+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01609
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00112
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ella

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss100000000.0ls0.05


# This is the optimal scaling parameters for this problem:
```
best3 = setup_simple_gb(
    length_scale=.05,
    scalar_scale=1e8 * 10,
)
```

In [16]:
best3 = setup_simple_gb(
    length_scale=.05,
    scalar_scale=1e8 * 10,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000000.0ls0.05
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:29:01+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01447
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00057
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ell

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000000.0ls0.05


In [17]:
checks(best2, best3)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


In [18]:
checks(best, best3)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


In [19]:
checks(unscl, best3)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


------------
# Playground for silly scaling combinations

In [20]:
scl_l1 = setup_simple_gb(
    length_scale=10,
    scalar_scale=pp.MEGA,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls10
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:29:10+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01748
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00120
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls10


In [21]:
scl_l2 = setup_simple_gb(
    length_scale=15,
    scalar_scale=pp.MEGA,
)

INFO:GTS.test.util:Path to results: /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls15
INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-03-31T14:29:19+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 14 intersections. Ellapsed time: 0.01702
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00122
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 56 points, 42
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed

INFO:GTS.isc_modelling.contact_mechanics_biot:'after_newton_convergence' returned None
INFO:GTS.isc_modelling.contact_mechanics_biot:Solution exported to folder 
 /home/haakon/mastersproject/src/mastersproject/GTS/test/results/test_parameter_scaling_on_regular_grid_ipynb/200331/test_1/ss1000000.0ls15


In [22]:
## Assert parameters on 3D grid ##
check_parameter_scaling(scl_l1, scl_l2, 3)

## Assert parameters on 2D grid ##
check_parameter_scaling(scl_l1, scl_l2, 2)

#

## Assert solutions on 3D grid ##
assert_solutions(scl_l1, scl_l2, 3)

## Assert solutions on 2D grid ##
assert_solutions(scl_l1, scl_l2, 2)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2
Checking solutions for two setups on grid of dimension: 3
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
p_exp: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
